## Energy-based Flow Classifier (EFC) - Pure Python

### Applying EFC to Bitcoin blockchain in the presence of label scarcity

In [8]:
from research_aml_elliptic.src.experiments.general_functions.elliptic_data_preprocessing import run_elliptic_preprocessing_pipeline

In [7]:
# Import Elliptic data set and set variables
last_time_step = 49
last_train_time_step = 34
only_labeled = True

In [18]:
X_train, X_test, y_train, y_test = run_elliptic_preprocessing_pipeline(last_train_time_step=last_train_time_step,
                                                                             last_time_step=last_time_step,
                                                                             only_labeled=only_labeled)

/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/research_aml_elliptic/src/experiments/general_functions/elliptic_data_preprocessing.py:33: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_classes.replace({'class': {'1': 1, '2': 0, 'unknown': 2}}, inplace=True)


In [32]:
import numpy as np
from sklearn.metrics import confusion_matrix

In [19]:
from efc_python.classification_functions import (
    one_class_fit,
    one_class_predict,
)
from efc_python.generic_discretize import discretize, get_intervals

In [20]:
intervals = get_intervals(X_train, 10)  # get discretization intervals from train set

In [21]:
X_train = discretize(X_train, intervals)  # discretize train
X_test = discretize(X_test, intervals)  # discretize test

/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/efc_python/generic_discretize.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data.iloc[:, feature].fillna(len(intervals[feature]), inplace=True)
/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/efc_python/generic_discretize.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '29894   NaN
29895   NaN
29896   NaN
29897   NaN
29898   NaN
         ..


In [24]:
idx_abnormal = np.where(y_train == 1)[0]  # find abnormal samples indexes in the training set

In [25]:
X_train.drop(idx_abnormal, axis=0, inplace=True)  # remove abnormal samples from training (EFC trains with only benign instances)

In [26]:
y_train.drop(idx_abnormal, axis=0, inplace=True)  # remove the corresponding abonrmal training targets

In [28]:
# EFC's hyperparameters
Q = X_test.values.max()
LAMBDA = 0.5  # pseudocount parameter

In [29]:
coupling, h_i, cutoff, _, _ = one_class_fit(np.array(X_train), Q, LAMBDA)  # train model

In [30]:
y_predicted, energies = one_class_predict(np.array(X_test), coupling, h_i, cutoff, Q)  # test model

In [33]:
# colect results
print("Single-class results")
print('confusion_matrix', confusion_matrix(y_test, y_predicted))

Single-class results
confusion_matrix [[10278  5309]
 [  857   226]]
